In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import multiprocessing

In [ ]:
paramsList = [3,1,15,5]

In [ ]:
parameters=["200 sma", "100 sma", "50 sma", "20 sma", "10 sma", "current price", "price 1 days after", "price 2 days after", "price 3 days after", "price 4 days after", "price 5 days after", "price 6 days after", "price 7 days after", "price 8 days after", "price 9 days after", "price 10 days after"]

openAverages=[]
for k in range(10,1205,5):
    openAverages.append(pd.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv"))
stocks=list(openAverages[0].columns)

start=0
end=239

total=0
momentumWorks=0
averagePercent=0

for k in range(start, end):
    openAverages[0]['AA'] = 
#     for j in range(len(stocks)):
#         if openAverages[k][paramsList[0],stocks[j]] > openAverages[k][paramsList[1],stocks[j]]:
#             total+=1
#             averagePercent+=((openAverages[k][paramsList[2],stocks[j]])/(openAverages[k][paramsList[3],stocks[j]]))
#             if openAverages[k][paramsList[2],stocks[j]] > openAverages[k][paramsList[3],stocks[j]]:
#                 momentumWorks+=1
try:
    averagePercent=averagePercent/total
except:
    averagePercent=1

days = abs(paramsList[3]-paramsList[2])
perDay = (averagePercent**(1/days))

if perDay>1.0000:

    tempParams=''
    for i in paramsList:
        tempParams+=str(i)

    tempRead=[]
    tempRead.append(parameters[paramsList[0]] + " is higher than " + parameters[paramsList[1]] + " " + str(total) + " times.")
    tempRead.append("Of these times, the " + parameters[paramsList[2]] + " is higher than the " + parameters[paramsList[3]] + " " + str(momentumWorks) + " times.")
    tempRead.append("Overall this is " + str(momentumWorks/total) + " of the time.")
    tempRead.append('The average increase is '+(str(averagePercent))+".")
    tempRead.append('This is '+str(perDay)+' increase per day.')
    # with open('../Data/sp500/bestStrategies/Single/Close/'+tempParams+'.txt','w') as file:
    #     for k in range(len(tempRead)):
    #         file.write(tempRead[k]+'\n')
openAverages[0]

In [224]:
openPdAverages=[]
for k in range(10,1205,5):
    openPdAverages.append(pd.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv"))
stocks=list(openPdAverages[0].columns)
openPdAverages[0]

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,136.710000,14.095000,131.670000,154.705000,146.410000,161.680000,103.450000,63.510000,277.065000,358.720000,...,93.385000,66.445000,107.480000,34.855000,104.035000,127.315000,124.970000,279.795000,40.910000,160.745000
1,130.110000,14.500000,108.750000,167.660000,145.850000,167.410000,103.400000,70.390000,281.700000,389.960000,...,107.060000,65.810000,108.270000,38.320000,104.040000,132.640000,132.130000,287.510000,31.430000,170.200000
2,122.360000,14.920000,91.580000,178.700000,138.420000,176.600000,105.780000,72.620000,294.380000,419.640000,...,105.020000,64.700000,105.900000,38.940000,106.660000,135.200000,136.860000,275.760000,26.640000,172.200000
3,118.050000,16.600000,67.750000,186.800000,134.150000,186.100000,105.150000,71.700000,307.900000,482.400000,...,103.850000,62.350000,104.600000,38.550000,110.850000,135.400000,141.700000,283.050000,27.600000,168.400000
4,117.800000,17.300000,68.500000,189.600000,133.000000,190.000000,106.500000,72.700000,304.300000,485.500000,...,103.700000,62.000000,104.800000,38.800000,110.400000,135.100000,142.700000,290.100000,27.300000,169.300000
5,118.480003,18.799999,69.872231,188.080002,134.083542,193.220001,106.986900,75.919998,308.600006,504.739990,...,108.260002,63.119999,105.970001,40.480000,112.660004,134.169998,143.179993,309.160004,30.430000,170.139999
6,120.000000,18.570000,70.500000,189.770004,133.500000,190.800003,106.340004,76.379997,313.880005,507.359985,...,108.870003,63.790001,106.489998,40.830002,111.620003,135.100006,142.649994,315.929993,31.280001,171.860001
7,119.349998,18.450001,70.660004,190.539993,133.589996,191.279999,107.739998,76.879997,315.209991,517.280029,...,109.639999,64.330002,104.540001,41.430000,110.750000,135.949997,143.110001,318.470001,32.000000,171.000000
8,119.750000,18.139999,70.639999,190.690002,136.009995,191.419998,107.930000,77.000000,315.769989,514.830017,...,109.510002,64.269997,100.940002,40.910000,109.449997,135.559998,142.369995,309.279999,30.700001,172.190002
9,118.250000,18.180000,69.480003,193.990005,134.830002,192.139999,107.000000,80.610001,319.779999,522.000000,...,107.019997,63.209999,101.379997,40.570000,110.629997,135.740005,140.660004,309.510010,31.360001,170.729996


In [ ]:
openPlAverages=[]
for k in range(10,1205,5):
    openPlAverages.append(pl.read_csv("../Data/sp500/sma/2017/Close/SimpleMovingAverages"+str(k)+".csv", infer_schema_length=None))
stocks=list(openAverages[0].columns)

In [225]:
test=openPdAverages
current=test[0]
for k in range(0, 239):
    for j in range(len(stocks)):
        if openAverages[k][paramsList[0],stocks[j]] > openAverages[k][paramsList[1],stocks[j]]:
            total+=1
            averagePercent+=((openAverages[k][paramsList[2],stocks[j]])/(openAverages[k][paramsList[3],stocks[j]]))
            if openAverages[k][paramsList[2],stocks[j]] > openAverages[k][paramsList[3],stocks[j]]:
                momentumWorks+=1

#mask = current.iloc[5] > current.iloc[4]
current.iloc[16] = current.iloc[5] > current.iloc[4]
current.iloc[16] = current.iloc[16].astype(int)

current

IndexError: iloc cannot enlarge its target object

In [219]:
#%%timeit -n25
current.iloc[16] = (current.iloc[5] > current.iloc[4]).astype(int)
sum(current.iloc[16])

IndexError: iloc cannot enlarge its target object

In [247]:
%%timeit -n1
#Pandas
import pandas as pd
import pickle
averagePercent=0
momentumWorks=0
total=0
test=openPdAverages
for k in range(0, 239):
    current=test[k]
    for j in range(len(stocks)):
        if current[stocks[j]][paramsList[0]] > current[stocks[j]][paramsList[1]]:
            total+=1
            averagePercent+=((current[stocks[j]][paramsList[2]])/(current[stocks[j]][paramsList[3]]))
            if current[stocks[j]][paramsList[2]] > current[stocks[j]][paramsList[3]]:
                momentumWorks+=1

2.04 s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [233]:
%%timeit -n1
import polars
averagePercent=0
momentumWorks=0
total=0
test=openPlAverages
for k in range(0, 239):
    current=test[k]
    for j in range(len(stocks)):
        if current[paramsList[0],stocks[j]] > current[paramsList[1],stocks[j]]:
            total+=1
            averagePercent+=((current[paramsList[2],stocks[j]])/(current[paramsList[3],stocks[j]]))
            if current[paramsList[2],stocks[j]] > current[paramsList[3],stocks[j]]:
                momentumWorks+=1

3.12 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
